In [1]:
import pandas as pd
import numpy as np
from scipy.stats import entropy
from sklearn.metrics.pairwise import haversine_distances

class TourismDiversityIndex:
    def __init__(self, coord_path, raw_tour_path, region):

        self.coord_path = coord_path
        self.raw_tour_path = raw_tour_path
        self.region = region

        self.use_full_index = region in ["경기"]

        self.rename_map = {
            "관광자원": "자연", "자연관광지": "자연",
            "레포츠소개": "레포츠", "수상 레포츠": "레포츠",
            "육상 레포츠": "레포츠", "복합 레포츠": "레포츠",
            "항공 레포츠": "레포츠",
            "휴양관광지": "휴양관광지", "체험관광지": "체험관광지",
            "건축/조형물": "건축/조형물", "문화시설": "문화시설",
            "산업관광지": "산업관광지", "역사관광지": "역사관광지",
            "음식점": "음식", "쇼핑": "쇼핑", "숙박시설": "숙박"
        }

        self.adjusted_df = None
        self.final_df = None

    def load_data(self):

        self.coord_df = pd.read_csv(self.coord_path)
        self.tour_df = pd.read_csv(self.raw_tour_path)


        if self.use_full_index:
            self.coord_df['행정동_full'] = self.coord_df['시군구명'] + '_' + self.coord_df['행정동명']
            self.coord_df = self.coord_df.set_index('행정동_full')

    def preprocess_tour_data(self):

        self.tour_df['cat3'] = self.tour_df['cat2'].map(self.rename_map)


        if self.region in ["경기"]:
            self.tour_df = self.tour_df.rename(columns={'ADM_CD': '행정구역_코드', 'ADM_NM': '행정동명'})
            merged = pd.merge(
                self.tour_df,
                self.coord_df[['행정동명', '행정구역_코드', '시군구명']],
                on=['행정동명', '행정구역_코드'],
                how='left'
            )
            self.tour_df['자치구'] = merged['시군구명']
            self.tour_df = self.tour_df.rename(columns={'행정동명': '행정동'})


        else:
            self.tour_df = self.tour_df.rename(columns={'ADM_NM': '행정동명'})

            coord_df_unique = self.coord_df.drop_duplicates(subset='행정동명', keep='first')

            merged = pd.merge(
                self.tour_df,
                coord_df_unique[['행정동명', '시군구명']],
                on='행정동명',
                how='left'
            )

            self.tour_df['자치구'] = merged['시군구명']

            self.tour_df = self.tour_df.rename(columns={'행정동명': '행정동'})

    def compute_entropy_index(self):

        grouped = self.tour_df[['자치구', '행정동', 'cat3']].value_counts().reset_index(name='개수')
        pivot = grouped.pivot(index=['자치구', '행정동'], columns='cat3', values='개수').fillna(0).astype(int)
        pivot.columns.name = None
        self.result_df = pivot.reset_index()


        self.result_df['총합'] = self.result_df.drop(columns=['자치구', '행정동']).sum(axis=1)

        numerical_df = self.result_df.drop(columns=['자치구', '행정동', '총합'])

        def entropy_score(row):
            total = row.sum()
            if total == 0:
                return 0
            probs = row / total
            return entropy(probs)

        self.result_df['체험다양성지수'] = numerical_df.apply(entropy_score, axis=1)

        self.content_df_gu = self.result_df[['자치구', '행정동']].copy()


        if self.use_full_index:
            self.result_df['행정동_full'] = self.result_df['자치구'] + '_' + self.result_df['행정동']
            self.result_df = self.result_df.set_index('행정동_full').drop(columns=['자치구', '행정동'])

    def match_and_filter_data(self):

        if self.use_full_index:
            self.coord_df['행정동_full'] = self.coord_df['시군구명'] + '_' + self.coord_df['행정동명']
            self.coord_df = self.coord_df.set_index('행정동_full').drop(columns=['시군구명', '행정동명'])

            self.result_df['행정동_full'] = self.result_df.index
            self.result_df = self.result_df.set_index('행정동_full')

            common_dongs = self.coord_df.index.intersection(self.result_df.index)
            self.coord_df = self.coord_df.loc[common_dongs]
            self.result_df = self.result_df.loc[common_dongs]

            self.content_df_gu = self.result_df.copy().reset_index()
            self.content_df_gu[['자치구', '행정동']] = self.content_df_gu['행정동_full'].str.split('_', n=1, expand=True)
            self.content_df_gu = self.content_df_gu[['자치구', '행정동']]
        else:
            self.coord_df = self.coord_df[self.coord_df['행정동명'].isin(self.result_df['행정동'])].reset_index(drop=True)
            self.result_df = self.result_df[self.result_df['행정동'].isin(self.coord_df['행정동명'])]
            self.content_df_gu = self.result_df[['자치구', '행정동']].copy()
            self.result_df = self.result_df.set_index('행정동').drop(columns='자치구')

    def calculate_distance_matrix(self):

        coords_rad = np.radians(self.coord_df[['위도', '경도']].values)
        dist_km = haversine_distances(coords_rad) * 6371
        return dist_km

    def create_weight_matrix(self, dist_km, threshold_km=None, epsilon=1e-5):

        n = dist_km.shape[0]
        p_matrix = np.zeros_like(dist_km)

        if threshold_km is None:
            if self.region == "서울":
                threshold_km = 2.5
            else:
                threshold_km = 13

        total_series = self.result_df['총합']
        entropy_series = self.result_df['체험다양성지수']

        q_total_75 = total_series.quantile(0.75)
        q_total_50 = total_series.quantile(0.5)
        q_entropy_75 = entropy_series.quantile(0.75)
        q_entropy_50 = entropy_series.quantile(0.5)

        for i in range(n):
            for j in range(n):
                if i == j:

                    total = total_series.iloc[i]
                    entropy = entropy_series.iloc[i]
                    w1 = 0.3 if total >= q_total_75 else 0.2 if total >= q_total_50 else 0.1
                    w2 = 0.3 if entropy >= q_entropy_75 else 0.2 if entropy >= q_entropy_50 else 0.1
                    p_matrix[i, j] = w1 + w2
                else:

                    d = dist_km[i, j]
                    p_matrix[i, j] = 1 / (2 * d + epsilon) if d < threshold_km else 0

        row_sums = p_matrix.sum(axis=1, keepdims=True)
        w_matrix = p_matrix / np.where(row_sums == 0, 1, row_sums)
        return w_matrix

    def calculate_adjusted_index(self, w_matrix):

        if self.use_full_index:
            weight_df = pd.DataFrame(w_matrix, index=self.coord_df.index, columns=self.coord_df.index)
        else:
            weight_df = pd.DataFrame(w_matrix, index=self.coord_df['행정동명'], columns=self.coord_df['행정동명'])

        adjusted_df = weight_df @ self.result_df


        if self.use_full_index:
            adjusted_df = adjusted_df.reset_index()
            adjusted_df[['자치구', '행정동']] = adjusted_df['행정동_full'].str.split('_', n=1, expand=True)
            adjusted_df = adjusted_df.drop(columns='행정동_full')
        else:
            adjusted_df.index.name = '행정동'
            adjusted_df = adjusted_df.reset_index().merge(self.content_df_gu, on='행정동', how='left')

        adjusted_df['힐링'] = adjusted_df['자연'] + adjusted_df['휴양관광지']
        adjusted_df['액티비티'] = adjusted_df['레포츠'] + adjusted_df['체험관광지']
        adjusted_df['문화'] = (
            adjusted_df['건축/조형물'] + adjusted_df['문화시설'] + adjusted_df['산업관광지'] + adjusted_df['역사관광지']
        )

        if self.use_full_index:
            adjusted_df = adjusted_df[['자치구', '행정동', '힐링', '액티비티', '문화', '음식', '쇼핑', '숙박']]
        else:
            adjusted_df = adjusted_df[['자치구', '행정동', '힐링', '액티비티', '문화', '음식', '쇼핑']]

        return adjusted_df

    def compute_entropy_std(self, adjusted_df):

        category_cols = adjusted_df.columns.difference(['자치구', '행정동'])

        def entropy_score(row):
            total = row.sum()
            if total == 0:
                return 0
            probs = row / total
            return entropy(probs)

        adjusted_df['체험다양성_엔트로피'] = adjusted_df[category_cols].apply(entropy_score, axis=1)
        adjusted_df['체험다양성_표준편차'] = adjusted_df[category_cols].std(axis=1)

        return adjusted_df

    def run_all(self):

        self.load_data()
        self.preprocess_tour_data()
        self.compute_entropy_index()
        self.match_and_filter_data()
        dist_km = self.calculate_distance_matrix()
        w_matrix = self.create_weight_matrix(dist_km)
        adjusted_df = self.calculate_adjusted_index(w_matrix)
        final_df = self.compute_entropy_std(adjusted_df)

        self.adjusted_df = adjusted_df
        self.final_df = final_df

    @property
    def adjusted_scores(self):
        if self.adjusted_df is None:
            self.run_all()
        return self.adjusted_df.copy()

    @property
    def diversity_entropy(self):
        if self.final_df is None:
            self.run_all()
        df = self.final_df[['자치구', '행정동', '체험다양성_엔트로피']].copy()
        return df.sort_values(by='체험다양성_엔트로피', ascending=False).reset_index(drop=True)

    @property
    def diversity_std(self):
        if self.final_df is None:
            self.run_all()
        df = self.final_df[['자치구', '행정동', '체험다양성_표준편차']].copy()
        return df.sort_values(by='체험다양성_표준편차', ascending=False).reset_index(drop=True)

    def get_category_score(self, category_name):
        if self.adjusted_df is None:
            self.run_all()
        if category_name in self.adjusted_df.columns:
            df = self.adjusted_df[['자치구', '행정동', category_name]].copy()
            return df.sort_values(by=category_name, ascending=False).reset_index(drop=True)
        else:
            raise ValueError(f"'{category_name}' 카테고리는 존재하지 않습니다.")

    def export(self, filename):
        if self.final_df is None:
            self.run_all()
        self.final_df.to_csv(filename, index=False)

# 출력 예시

In [2]:
ex = TourismDiversityIndex("경기_중심좌표.csv", "경기_중심좌표_행정동_매핑_Final.csv", "경기")
adjusted = ex.adjusted_scores
print(adjusted.head())

   자치구  행정동         힐링       액티비티        문화         음식        쇼핑        숙박  \
0  가평군  가평읍   8.876844  35.535411  8.340898  32.220430  0.889715  0.000000   
1  가평군   북면  16.513754  39.817658  9.000000   5.884205  0.060781  0.000000   
2  가평군   상면   6.450964  25.268229  3.985214  17.605723  0.133293  0.000000   
3  가평군  설악면   8.757376  47.974015  4.162964  28.523809  0.939065  0.000000   
4  가평군  조종면   2.879345  20.849044  3.314121  16.775161  0.733022  0.058038   

   체험다양성_엔트로피  체험다양성_표준편차  
0    1.241378   15.629019  
1    1.137316   15.013705  
2    1.183715   10.294914  
3    1.115577   19.231637  
4    1.169467    8.991628  


In [3]:
healing_score = ex.get_category_score("힐링")
print(healing_score.head())

       자치구  행정동         힐링
0  안산시 단원구  대부동  20.403651
1      가평군   북면  16.513754
2      화성시  서신면  11.718068
3      가평군  가평읍   8.876844
4      가평군  설악면   8.757376


In [4]:
activity_score = ex.get_category_score("액티비티")
print(activity_score.head())

       자치구  행정동       액티비티
0      가평군  설악면  47.974015
1      가평군   북면  39.817658
2  안산시 단원구  대부동  36.508096
3      가평군  가평읍  35.535411
4      포천시  이동면  28.333468


In [5]:
culture_score = ex.get_category_score("문화")
print(culture_score.head())

       자치구  행정동         문화
0      파주시  탄현면  15.033604
1  안산시 단원구  대부동   9.259537
2      가평군   북면   9.000000
3      가평군  가평읍   8.340898
4      파주시  문산읍   8.112121


In [ ]:
food_score = ex.get_category_score("음식")
print(food_score.head())

       자치구  행정동         음식
0  안산시 단원구  대부동  42.797192
1      가평군  가평읍  32.220430
2      가평군  청평면  30.875514
3      가평군  설악면  28.523809
4      파주시  탄현면  22.027206


In [6]:
shopping_score = ex.get_category_score("쇼핑")
print(shopping_score.head())

   자치구   행정동         쇼핑
0  파주시   탄현면  49.006621
1  여주시   여흥동  42.078555
2  이천시   호법면  37.580888
3  김포시   고촌읍  15.937002
4  파주시  운정5동  14.986075


In [7]:
stay_score = ex.get_category_score("숙박")
print(stay_score.head())

   자치구  행정동        숙박
0  포천시  일동면  0.629838
1  포천시  영중면  0.089668
2  포천시  이동면  0.070447
3  포천시  화현면  0.060142
4  가평군  조종면  0.058038


In [8]:
diversity_entropy = ex.diversity_entropy
print(diversity_entropy.head())

       자치구  행정동  체험다양성_엔트로피
0  안산시 단원구  초지동    1.547459
1      여주시  오학동    1.545467
2  용인시 처인구  백암면    1.541996
3      화성시  비봉면    1.538306
4      여주시  북내면    1.518434


In [9]:
diversity_std = ex.diversity_std
print(diversity_std.head())

       자치구  행정동  체험다양성_표준편차
0      가평군  설악면   19.231637
1      파주시  탄현면   18.267681
2  안산시 단원구  대부동   18.176786
3      여주시  여흥동   15.771402
4      가평군  가평읍   15.629019


In [10]:
ex = TourismDiversityIndex('경기_중심좌표.csv', '경기_중심좌표_행정동_매핑_Final.csv','경기')
ex.export('경기_관광지수_최종.csv')

In [11]:
ex = TourismDiversityIndex('서울_중심좌표.csv', '서울_중심좌표_행정동_매핑_Final.csv','서울')
ex.export('서울_관광지수_최종.csv')

In [12]:
ex = TourismDiversityIndex('인천_중심좌표.csv', '인천_중심좌표_행정동_매핑_Final.csv','인천')
ex.export('인천_관광지수_최종.csv')

# 주석 처리 (코드 설명 포함 ) 버전

In [13]:
df1 = pd.read_csv('경기_관광지수_최종.csv')
df2 = pd.read_csv('서울_관광지수_최종.csv')
df3 = pd.read_csv('인천_관광지수_최종.csv')


In [14]:
df_merged = pd.concat([df1, df2, df3], axis=0)

In [15]:
df_merged

,자치구,행정동,힐링,액티비티,문화,음식,쇼핑,숙박,체험다양성_엔트로피,체험다양성_표준편차
0,가평군,가평읍,8.876844,35.535411,8.340898,32.220430,0.889715,0.000000,1.241378,15.629019
1,가평군,북면,16.513754,39.817658,9.000000,5.884205,0.060781,0.000000,1.137316,15.013705
2,가평군,상면,6.450964,25.268229,3.985214,17.605723,0.133293,0.000000,1.183715,10.294914
3,가평군,설악면,8.757376,47.974015,4.162964,28.523809,0.939065,0.000000,1.115577,19.231637
4,가평군,조종면,2.879345,20.849044,3.314121,16.775161,0.733022,0.058038,1.169467,8.991628
...,...,...,...,...,...,...,...,...,...,...
135,중구,영종1동,1.378579,1.261023,1.519679,4.880436,0.436482,NaN,1.325857,1.721161
136,중구,영종2동,1.337295,1.182559,1.417489,4.793340,0.437261,NaN,1.317795,1.699496
137,중구,영종동,2.213174,2.429071,1.643989,5.732951,0.347635,NaN,1.352629,1.993813
138,중구,용유동,23.811770,11.947058,2.922033,37.514162,0.000000,NaN,1.127921,15.527773


In [16]:
# CSV 파일로 저장
df_merged.to_csv("final_data_Tourism.csv", index=False, encoding="utf-8-sig")


In [ ]:
# import pandas as pd
# import numpy as np
# from scipy.stats import entropy
# from sklearn.metrics.pairwise import haversine_distances

# class TourismDiversityIndex:
#     def __init__(self, coord_path, raw_tour_path, region):

#         # 파일 경로와 지역 설정
#         self.coord_path = coord_path        # 행정동별 좌표 데이터
#         self.raw_tour_path = raw_tour_path  # 관광지 데이터
#         self.region = region                # 대상 지역 ( 경기 or 인천 or 서울)

#         self.use_full_index = region in ["경기"]   # 경기 지역이면 '자치구_행정동'을 인덱스로 사용 ( 행정동 이름 중복 막기 위함 )

#         # 카테고리명 표준화 매핑
#         self.rename_map = {
#             "관광자원": "자연", "자연관광지": "자연",
#             "레포츠소개": "레포츠", "수상 레포츠": "레포츠",
#             "육상 레포츠": "레포츠", "복합 레포츠": "레포츠",
#             "항공 레포츠": "레포츠",
#             "휴양관광지": "휴양관광지", "체험관광지": "체험관광지",
#             "건축/조형물": "건축/조형물", "문화시설": "문화시설",
#             "산업관광지": "산업관광지", "역사관광지": "역사관광지",
#             "음식점": "음식", "쇼핑": "쇼핑", "숙박시설": "숙박"
#         }
#         # 필요한 속성 초기화
#         self.adjusted_df = None
#         self.final_df = None

#     def load_data(self):
#         # 좌표와 관광지 데이터 불러오기
#         self.coord_df = pd.read_csv(self.coord_path)
#         self.tour_df = pd.read_csv(self.raw_tour_path)

#         # 경기 지역은 행정동_full 컬럼 생성 후 인덱스 지정
#         if self.use_full_index:
#             self.coord_df['행정동_full'] = self.coord_df['시군구명'] + '_' + self.coord_df['행정동명']
#             self.coord_df = self.coord_df.set_index('행정동_full')

#     def preprocess_tour_data(self):
#         # 기본 설정된 관광지 유형(cat2)을 변경한 카테고리(cat3)로 변환
#         self.tour_df['cat3'] = self.tour_df['cat2'].map(self.rename_map)

#         # 경기 지역은 자치구/행정동/행정구역코드 기준 병합 처리
#         if self.region in ["경기"]:
#             self.tour_df = self.tour_df.rename(columns={'ADM_CD': '행정구역_코드', 'ADM_NM': '행정동명'})
#             merged = pd.merge(
#                 self.tour_df,
#                 self.coord_df[['행정동명', '행정구역_코드', '시군구명']],
#                 on=['행정동명', '행정구역_코드'],
#                 how='left'
#             )
#             self.tour_df['자치구'] = merged['시군구명']
#             self.tour_df = self.tour_df.rename(columns={'행정동명': '행정동'})

#         # 나머지는 행정동 기준 병합 처리
#         else:
#             self.tour_df = self.tour_df.rename(columns={'ADM_NM': '행정동명'})

#             coord_df_unique = self.coord_df.drop_duplicates(subset='행정동명', keep='first')

#             merged = pd.merge(
#                 self.tour_df,
#                 coord_df_unique[['행정동명', '시군구명']],
#                 on='행정동명',
#                 how='left'
#             )

#             self.tour_df['자치구'] = merged['시군구명']

#             self.tour_df = self.tour_df.rename(columns={'행정동명': '행정동'})

#     def compute_entropy_index(self):
#         # 카테고리별 관광지 개수 집계
#         grouped = self.tour_df[['자치구', '행정동', 'cat3']].value_counts().reset_index(name='개수')
#         pivot = grouped.pivot(index=['자치구', '행정동'], columns='cat3', values='개수').fillna(0).astype(int)
#         pivot.columns.name = None
#         self.result_df = pivot.reset_index()

#         # 총 관광지 개수와 엔트로피 계산
#         self.result_df['총합'] = self.result_df.drop(columns=['자치구', '행정동']).sum(axis=1)

#         numerical_df = self.result_df.drop(columns=['자치구', '행정동', '총합'])

#         def entropy_score(row):
#             total = row.sum()
#             if total == 0:
#                 return 0
#             probs = row / total
#             return entropy(probs)

#         self.result_df['체험다양성지수'] = numerical_df.apply(entropy_score, axis=1)

#         self.content_df_gu = self.result_df[['자치구', '행정동']].copy()

#         # 경기 지역이면 인덱스를 행정동_full로 변경
#         if self.use_full_index:
#             self.result_df['행정동_full'] = self.result_df['자치구'] + '_' + self.result_df['행정동']
#             self.result_df = self.result_df.set_index('행정동_full').drop(columns=['자치구', '행정동'])

#     def match_and_filter_data(self):
#         # 좌표 데이터와 관광지 데이터를 공통 행정동 기준으로 정렬 및 정제
#         if self.use_full_index:
#             self.coord_df['행정동_full'] = self.coord_df['시군구명'] + '_' + self.coord_df['행정동명']
#             self.coord_df = self.coord_df.set_index('행정동_full').drop(columns=['시군구명', '행정동명'])

#             self.result_df['행정동_full'] = self.result_df.index
#             self.result_df = self.result_df.set_index('행정동_full')

#             common_dongs = self.coord_df.index.intersection(self.result_df.index)
#             self.coord_df = self.coord_df.loc[common_dongs]
#             self.result_df = self.result_df.loc[common_dongs]

#             self.content_df_gu = self.result_df.copy().reset_index()
#             self.content_df_gu[['자치구', '행정동']] = self.content_df_gu['행정동_full'].str.split('_', n=1, expand=True)
#             self.content_df_gu = self.content_df_gu[['자치구', '행정동']]
#         else:
#             self.coord_df = self.coord_df[self.coord_df['행정동명'].isin(self.result_df['행정동'])].reset_index(drop=True)
#             self.result_df = self.result_df[self.result_df['행정동'].isin(self.coord_df['행정동명'])]
#             self.content_df_gu = self.result_df[['자치구', '행정동']].copy()
#             self.result_df = self.result_df.set_index('행정동').drop(columns='자치구')

#     def calculate_distance_matrix(self):
#         # 위도/경도를 라디안 단위로 변환한 후 하버사인 거리 행렬 계산
#         coords_rad = np.radians(self.coord_df[['위도', '경도']].values)
#         dist_km = haversine_distances(coords_rad) * 6371 # 지구 반지름을 곱 거리(km)
#         return dist_km

#     def create_weight_matrix(self, dist_km, threshold_km=None, epsilon=1e-5):
#         # 거리 기반 공간가중행렬 생성
#         n = dist_km.shape[0]
#         p_matrix = np.zeros_like(dist_km)

#         # 지역에 따라 거리 기준 임계값 설정
#         if threshold_km is None:
#             if self.region == "서울":
#                 threshold_km = 2.5
#             else:
#                 threshold_km = 13

#         # 관광지 총합과 체험다양성 기준으로 자기신뢰도(자기자신가중치) 설정
#         total_series = self.result_df['총합']
#         entropy_series = self.result_df['체험다양성지수']

#         q_total_75 = total_series.quantile(0.75)
#         q_total_50 = total_series.quantile(0.5)
#         q_entropy_75 = entropy_series.quantile(0.75)
#         q_entropy_50 = entropy_series.quantile(0.5)

#         for i in range(n):
#             for j in range(n):
#                 if i == j:
#                     # 자기 자신에 대한 가중치 (관광지수 + 엔트로피)
#                     total = total_series.iloc[i]
#                     entropy = entropy_series.iloc[i]
#                     w1 = 0.3 if total >= q_total_75 else 0.2 if total >= q_total_50 else 0.1
#                     w2 = 0.3 if entropy >= q_entropy_75 else 0.2 if entropy >= q_entropy_50 else 0.1
#                     p_matrix[i, j] = w1 + w2
#                 else:
#                     # 인접 행정동에 대한 거리 기반 가중치
#                     d = dist_km[i, j]
#                     p_matrix[i, j] = 1 / (2 * d + epsilon) if d < threshold_km else 0

#         # 행 단위 정규화 (총합 1)
#         row_sums = p_matrix.sum(axis=1, keepdims=True)
#         w_matrix = p_matrix / np.where(row_sums == 0, 1, row_sums)
#         return w_matrix

#     def calculate_adjusted_index(self, w_matrix):
#         # 공간가중행렬을 곱해 공간보정지수 계산
#         if self.use_full_index:
#             weight_df = pd.DataFrame(w_matrix, index=self.coord_df.index, columns=self.coord_df.index)
#         else:
#             weight_df = pd.DataFrame(w_matrix, index=self.coord_df['행정동명'], columns=self.coord_df['행정동명'])

#         # 행정동별 보정 관광지 수 계산
#         adjusted_df = weight_df @ self.result_df

#         # 결과 포맷 정리
#         if self.use_full_index:
#             adjusted_df = adjusted_df.reset_index()
#             adjusted_df[['자치구', '행정동']] = adjusted_df['행정동_full'].str.split('_', n=1, expand=True)
#             adjusted_df = adjusted_df.drop(columns='행정동_full')
#         else:
#             adjusted_df.index.name = '행정동'
#             adjusted_df = adjusted_df.reset_index().merge(self.content_df_gu, on='행정동', how='left')

#         # 관광지 유형을 카테고리별로 묶기
#         adjusted_df['힐링'] = adjusted_df['자연'] + adjusted_df['휴양관광지']
#         adjusted_df['액티비티'] = adjusted_df['레포츠'] + adjusted_df['체험관광지']
#         adjusted_df['문화'] = (
#             adjusted_df['건축/조형물'] + adjusted_df['문화시설'] + adjusted_df['산업관광지'] + adjusted_df['역사관광지']
#         )

#         # 결과 컬럼 선택
#         if self.use_full_index:
#             adjusted_df = adjusted_df[['자치구', '행정동', '힐링', '액티비티', '문화', '음식', '쇼핑', '숙박']]
#         else:
#             adjusted_df = adjusted_df[['자치구', '행정동', '힐링', '액티비티', '문화', '음식', '쇼핑']]

#         return adjusted_df

#     def compute_entropy_std(self, adjusted_df):
#         # 공간보정지수 기반 엔트로피와 표준편차 계산
#         category_cols = adjusted_df.columns.difference(['자치구', '행정동'])

#         def entropy_score(row):
#             total = row.sum()
#             if total == 0:
#                 return 0
#             probs = row / total
#             return entropy(probs)

#         adjusted_df['체험다양성_엔트로피'] = adjusted_df[category_cols].apply(entropy_score, axis=1)
#         adjusted_df['체험다양성_표준편차'] = adjusted_df[category_cols].std(axis=1)

#         return adjusted_df

#     def run_all(self):
#         # 전체 흐름 실행
#         self.load_data()
#         self.preprocess_tour_data()
#         self.compute_entropy_index()
#         self.match_and_filter_data()
#         dist_km = self.calculate_distance_matrix()
#         w_matrix = self.create_weight_matrix(dist_km)
#         adjusted_df = self.calculate_adjusted_index(w_matrix)
#         final_df = self.compute_entropy_std(adjusted_df)

#         self.adjusted_df = adjusted_df
#         self.final_df = final_df

#     @property
#     def adjusted_scores(self): # 공간보정지수 전체 DataFrame 반환
#         if self.adjusted_df is None:
#             self.run_all()
#         return self.adjusted_df.copy()

#     @property
#     def diversity_entropy(self): # 체험다양성_엔트로피 기준 내림차순 정렬 결과 반환
#         if self.final_df is None:
#             self.run_all()
#         df = self.final_df[['자치구', '행정동', '체험다양성_엔트로피']].copy()
#         return df.sort_values(by='체험다양성_엔트로피', ascending=False).reset_index(drop=True)

#     @property
#     def diversity_std(self): # 체험다양성_표준편차 기준 내림차순 정렬 결과 반환
#         if self.final_df is None:
#             self.run_all()
#         df = self.final_df[['자치구', '행정동', '체험다양성_표준편차']].copy()
#         return df.sort_values(by='체험다양성_표준편차', ascending=False).reset_index(drop=True)

#     def get_category_score(self, category_name): # 지정 카테고리 점수 기준 내림차순 정렬 결과 반환
#         if self.adjusted_df is None:
#             self.run_all()
#         if category_name in self.adjusted_df.columns:
#             df = self.adjusted_df[['자치구', '행정동', category_name]].copy()
#             return df.sort_values(by=category_name, ascending=False).reset_index(drop=True)
#         else:
#             raise ValueError(f"'{category_name}' 카테고리는 존재하지 않습니다.")

#     def export(self, filename):  # 전체 결과 CSV로 저장
#         if self.final_df is None:
#             self.run_all()
#         self.final_df.to_csv(filename, index=False)